# **Generate a timelapse from existing video**  
  
### **Table of Contents**
1. [Set Paths](#paths)
2. [YOLO-Model](#model)
3. [Parameters](#params)
4. [Process video](#process_video)

In [39]:
from ultralytics import YOLO
import os
import cv2
import math

### **Set Paths** <a id="paths"></a>

`video` = path to the raw video of the printing video  
`weights` = best weights for yolo model achieved during training  
`outputFolder` = path to store the created timelapse

In [40]:
video = "C:\\Users\\Fabio\\Desktop\\Printhead_AI\\TestVideo2.mp4"
weights = "runs/detect/train11/weights/best.pt"
outputFolder = "C:\\Users\\Fabio\\Desktop\\TestOutput"

---

### **Import trained YOLO model** <a id="model"></a>
* currently using YOLOV8

In [41]:
model = YOLO(weights)

---

### **Set parameters**  <a id="params"></a>
* before processing we need to capture fps, set params for tracking and frame skipping
* `cv2.VideoCapture(video)`: replace the **input** with either a saved video or a live input from a webcam `cv2.VideoCapture(0)`

In [77]:
cap = cv2.VideoCapture(0)

# get raw vid fps
if cap.get(cv2.CAP_PROP_FPS) == 30:
    fSkip = 7.5
else:
    fSkip = math.ceil(cap.get(cv2.CAP_PROP_FPS)/4)

prevX = 1
lastWasTheSame = False
wiggleRoom = 0.003
frame_skip = fSkip
frame_count = 0
photo_taken = False
x = None

---

### **Process Video Frame-by-Frame** <a id="process_video"></a>
* The while loop reads the video frame by frame, detects the printhead's position, and saves the necessary images.

In [78]:
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    # extract dimensions once on first frame
    if frame_count == 0:
        hImg, wImg, _ = frame.shape

    if frame_count % frame_skip == 0:
        results = model.predict(frame, device = "cuda")
        frame_skip = fSkip

        for r in results:
            for box in r.boxes:
                x, y, w, h = box.xywh[0].tolist() # Get the x, y, w, h coordinates.
                xNorm, yNorm, wNorm, hNorm =  x/wImg, y/hImg, w/wImg, h/hImg
                print(xNorm)

                if abs(xNorm - prevX) < wiggleRoom:
                    prevX = xNorm
                    if not lastWasTheSame:
                        lastWasTheSame = True
                    elif not photo_taken:
                        filename = os.path.join(outputFolder, f"frame_{frame_count}.jpg")
                        cv2.imwrite(filename, frame)
                        photo_taken = True
                        
                else:
                    lastWasTheSame = False
                    photo_taken = False

                if prevX > xNorm :
                    prevX = xNorm
    frame_count += 1
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 Printheads, 8.3ms
Speed: 3.2ms preprocess, 8.3ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
0.3028695106506348

0: 384x640 1 Printheads, 6.6ms
Speed: 1.5ms preprocess, 6.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
0.30285460154215493

0: 384x640 1 Printheads, 10.9ms
Speed: 1.5ms preprocess, 10.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)
0.3028831164042155

0: 384x640 1 Printheads, 7.6ms
Speed: 1.9ms preprocess, 7.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
0.3028813680013021

0: 384x640 1 Printheads, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 4.6ms postprocess per image at shape (1, 3, 384, 640)
0.30286725362141925

0: 384x640 1 Printheads, 7.1ms
Speed: 1.6ms preprocess, 7.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
0.3028292655944824

0: 384x640 1 Printheads, 13.0ms
Speed: 1.8ms preprocess, 13.0ms inference, 3.5ms postprocess per image at shape (

KeyboardInterrupt: 